In [ ]:
import scvelo as scv
import scanpy as sc
import cell2fate as c2f
import pickle as pickle
from eval_utils import cross_boundary_correctness
from datetime import datetime
import pandas as pd
import numpy as np
from os.path import exists
import matplotlib.pyplot as plt
import torch
import unitvelo as utv
method = 'cell2fate'
data_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_datasets/'
save_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_results/'
dataset = 'Pancreas_with_cc'
n_genes_list = np.array((1500, 3000, 6000))
n_counts_list = np.array((5, 10, 20))
# Model params:
mod_list = [0.8, 1, 1.2]
# Robust Training Functions:
def infer_dynamics(mod, max_epochs = [100, 400], lr = [0.02, 0.01]):
    print('First training run ...')
    mod.train(max_epochs = max_epochs[0], lr = lr[0])
    print('Initializing second training run ...')
    sample_kwarg = {"num_samples": 3, "batch_size" : adata.n_obs,
                     "use_gpu" : True, 'return_samples': False}
    mod.adata = mod.export_posterior(mod.adata, sample_kwargs=sample_kwarg)
    init_vals = {}
    params = ['splicing_alpha', 'splicing_mean', 'beta_g', 'degredation_alpha', 'degredation_mean', 'gamma_g', 'factor_level_g',
              'g_fg', 'A_mgON', 'detection_mean_y_e', 'detection_hyp_prior_alpha', 'detection_y_c',
              'detection_y_i', 'detection_y_gi', 's_g_gene_add_alpha_hyp', 's_g_gene_add_mean', 's_g_gene_add_alpha_e_inv',
              's_g_gene_add']
    for i in range(len(params)):
            init_vals[params[i]] = mod.samples['post_sample_means'][params[i]]
    print('Second training run ...')
    mod2 = c2f.Cell2fate_DynamicalModel(mod.adata, n_modules = n_modules,
            init_vals = init_vals)
    del mod
    mod2.train(max_epochs = max_epochs[1], lr = lr[1])
    return mod2
for i in range(len(n_genes_list)):
    for j in range(len(n_counts_list)):
        for k in range(len(mod_list)):
                print(i)
                print(j)
                print(k)
                n_genes = n_genes_list[i]
                min_counts = n_counts_list[j]
                model_index = str(i) + '-' + str(j) + '-' + str(k)
                save_name = method + '_' + dataset
                if exists(save_dir + save_name + '_CBDC_And_Posterior.csv'):
                    tab = pd.read_csv(save_dir + save_name + '_CBDC_And_Posterior.csv', index_col = 0)
                    if model_index in tab.index:
                        continue
                adata = sc.read_h5ad(data_dir + dataset + '/' + dataset + '_anndata.h5ad')
                adata = c2f.utils.get_training_data(adata, cells_per_cluster = 10**5, cluster_column = 'clusters',
                                                remove_clusters = [], min_shared_counts = min_counts, n_var_genes= n_genes)
                c2f.Cell2fate_DynamicalModel.setup_anndata(adata, spliced_label='spliced', unspliced_label='unspliced')    
                n_modules = int(np.round(c2f.utils.get_max_modules(adata)*mod_list[k]))
                mod = c2f.Cell2fate_DynamicalModel(adata,
                                                   n_modules = n_modules)   
                mod = infer_dynamics(mod)
                sample_kwarg = {"num_samples": 3, "batch_size" : adata.n_obs,
                     "use_gpu" : True, 'return_samples': False}
                adata = mod.export_posterior(adata, sample_kwargs = sample_kwarg)
                mod.compute_and_plot_total_velocity(adata, save = False, delete = False)
                # Calculate performance metrics:
                file = open(data_dir + dataset + '/' + dataset + '_groundTruth.pickle' ,'rb')
                ground_truth = pickle.load(file)
                metrics = utv.evaluate(adata, ground_truth, 'clusters', 'Velocity')
                cb_score = [np.mean(metrics['Cross-Boundary Direction Correctness (A->B)'][x])
                            for x in metrics['Cross-Boundary Direction Correctness (A->B)'].keys()]
                if exists(save_dir + save_name + '_CBDC_And_Posterior.csv'):
                    tab = pd.read_csv(save_dir + save_name + '_CBDC_And_Posterior.csv', index_col = 0)
                else:
                    c_names = ['CBDC']
                    tab = pd.DataFrame(columns = c_names)
                tab.loc[model_index, 'CBDC'] = np.mean(cb_score)
                tab.loc[model_index, 'splicing_alpha'] = mod.samples['post_sample_means']['splicing_alpha']
                tab.loc[model_index, 'splicing_mean'] = mod.samples['post_sample_means']['splicing_mean']
                tab.loc[model_index, 'beta_g_mean'] = np.mean(mod.samples['post_sample_means']['beta_g'])
                tab.loc[model_index, 'beta_g_var'] = np.var(mod.samples['post_sample_means']['beta_g'])
                tab.loc[model_index, 'degredation_alpha'] = mod.samples['post_sample_means']['degredation_alpha']
                tab.loc[model_index, 'degredation_mean'] = mod.samples['post_sample_means']['degredation_mean']
                tab.loc[model_index, 'gamma_g_mean'] = np.mean(mod.samples['post_sample_means']['gamma_g'])
                tab.loc[model_index, 'gamma_g_var'] = np.var(mod.samples['post_sample_means']['gamma_g'])
                tab.loc[model_index, 'factor_level_g_mean'] = np.mean(mod.samples['post_sample_means']['factor_level_g'])
                tab.loc[model_index, 'factor_level_g_var'] = np.var(mod.samples['post_sample_means']['factor_level_g'])
                tab.loc[model_index, 'g_fg_mean'] = np.mean(mod.samples['post_sample_means']['g_fg'])
                tab.loc[model_index, 'g_fg_var'] = np.var(mod.samples['post_sample_means']['g_fg'])
                tab.loc[model_index, 'A_mgON_mean'] = np.mean(mod.samples['post_sample_means']['A_mgON'])
                tab.loc[model_index, 'A_mgON_var'] = np.var(mod.samples['post_sample_means']['A_mgON'])
                tab.loc[model_index, 'lam_mu'] = mod.samples['post_sample_means']['lam_mu']
                tab.loc[model_index, 'lam_sd'] = mod.samples['post_sample_means']['lam_sd']
                tab.loc[model_index, 'lam_m_mu_mean'] = np.mean(mod.samples['post_sample_means']['lam_m_mu'])
                tab.loc[model_index, 'lam_m_mu_var'] = np.var(mod.samples['post_sample_means']['lam_m_mu'])
                tab.loc[model_index, 'lam_mi_mean'] = np.mean(mod.samples['post_sample_means']['lam_mi'])
                tab.loc[model_index, 'lam_mi_var'] = np.var(mod.samples['post_sample_means']['lam_mi'])
                tab.loc[model_index, 'Tmax'] = mod.samples['post_sample_means']['Tmax']
                tab.loc[model_index, 't_c_loc'] = mod.samples['post_sample_means']['t_c_loc']
                tab.loc[model_index, 't_c_scale'] = mod.samples['post_sample_means']['t_c_scale']
                tab.loc[model_index, 'T_c_mean'] = np.mean(mod.samples['post_sample_means']['T_c'])
                tab.loc[model_index, 'T_c_var'] = np.var(mod.samples['post_sample_means']['T_c'])
                tab.loc[model_index, 't_delta_mean'] = np.mean(mod.samples['post_sample_means']['t_delta'])
                tab.loc[model_index, 't_delta_var'] = np.var(mod.samples['post_sample_means']['t_delta'])
                tab.loc[model_index, 'T_mON_mean'] = np.mean(mod.samples['post_sample_means']['T_mON'])
                tab.loc[model_index, 'T_mON_var'] = np.var(mod.samples['post_sample_means']['T_mON'])
                tab.loc[model_index, 'T_mON_mean'] = np.mean(mod.samples['post_sample_means']['T_mON'])
                tab.loc[model_index, 'T_mON_var'] = np.var(mod.samples['post_sample_means']['T_mON'])
                tab.loc[model_index, 'T_mOFF_mean'] = np.mean(mod.samples['post_sample_means']['T_mOFF'])
                tab.loc[model_index, 'T_mOFF_var'] = np.var(mod.samples['post_sample_means']['T_mOFF'])
                tab.loc[model_index, 'mu_expression_mean'] = np.mean(mod.samples['post_sample_means']['mu_expression'])
                tab.loc[model_index, 'mu_expression_var'] = np.var(mod.samples['post_sample_means']['mu_expression'])
                tab.loc[model_index, 'detection_mean_y_e_mean'] = np.mean(mod.samples['post_sample_means']['detection_mean_y_e'])
                tab.loc[model_index, 'detection_mean_y_e_var'] = np.var(mod.samples['post_sample_means']['detection_mean_y_e'])
                tab.loc[model_index, 'detection_hyp_prior_alpha'] = mod.samples['post_sample_means']['detection_hyp_prior_alpha']
                tab.loc[model_index, 'detection_y_c_mean'] = np.mean(mod.samples['post_sample_means']['detection_y_c'])
                tab.loc[model_index, 'detection_y_c_var'] = np.var(mod.samples['post_sample_means']['detection_y_c'])
                tab.loc[model_index, 'detection_y_i_mean'] = np.mean(mod.samples['post_sample_means']['detection_y_i'])
                tab.loc[model_index, 'detection_y_i_var'] = np.var(mod.samples['post_sample_means']['detection_y_i'])
                tab.loc[model_index, 'detection_y_gi_mean'] = np.mean(mod.samples['post_sample_means']['detection_y_gi'])
                tab.loc[model_index, 'detection_y_gi_var'] = np.var(mod.samples['post_sample_means']['detection_y_gi'])
                tab.loc[model_index, 's_g_gene_add_alpha_hyp_mean'] = np.mean(mod.samples['post_sample_means']['s_g_gene_add_alpha_hyp'])
                tab.loc[model_index, 's_g_gene_add_alpha_hyp_var'] = np.var(mod.samples['post_sample_means']['s_g_gene_add_alpha_hyp'])
                tab.loc[model_index, 's_g_gene_add_mean_mean'] = np.mean(mod.samples['post_sample_means']['s_g_gene_add_mean'])
                tab.loc[model_index, 's_g_gene_add_mean_var'] = np.var(mod.samples['post_sample_means']['s_g_gene_add_mean'])
                tab.loc[model_index, 's_g_gene_add_alpha_e_inv_mean'] = np.mean(mod.samples['post_sample_means']['s_g_gene_add_alpha_e_inv'])
                tab.loc[model_index, 's_g_gene_add_alpha_e_inv_var'] = np.var(mod.samples['post_sample_means']['s_g_gene_add_alpha_e_inv'])
                tab.loc[model_index, 's_g_gene_add_mean'] = np.mean(mod.samples['post_sample_means']['s_g_gene_add'])
                tab.loc[model_index, 's_g_gene_add_var'] = np.var(mod.samples['post_sample_means']['s_g_gene_add'])
                tab.loc[model_index, 'stochastic_v_ag_hyp_mean'] = np.mean(mod.samples['post_sample_means']['stochastic_v_ag_hyp'])
                tab.loc[model_index, 'stochastic_v_ag_hyp_var'] = np.var(mod.samples['post_sample_means']['stochastic_v_ag_hyp'])
                tab.loc[model_index, 'stochastic_v_ag_inv_mean'] = np.mean(mod.samples['post_sample_means']['stochastic_v_ag_inv'])
                tab.loc[model_index, 'stochastic_v_ag_inv_var'] = np.var(mod.samples['post_sample_means']['stochastic_v_ag_inv'])
                tab.to_csv(save_dir + save_name + '_CBDC_And_Posterior.csv')

Global seed set to 0


(Running UniTVelo 0.2.5)
2023-10-10 10:12:25
0
0
0
Keeping at most 100000 cells per cluster
Filtered out 18634 genes that are detected 5 counts (shared).
Extracted 1500 highly variable genes.
Leiden clustering ...
Number of Leiden Clusters: 13
Maximal Number of Modules: 14
First training run ...


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:51<00:00,  1.12s/it, v_num=1, elbo_train=5.06e+6]
Initializing second training run ...
Sampling global variables, sample: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.00it/s]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Second training run ...
Sampling global variables, sample: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.05it/s]
Computing total RNAvelocity ...
# Cross-Boundary Direction Correctness (A->B)
{('Ngn3 high EP', 'Pre-endocrine'): -0.8428453304983066, ('Pre-endocrine', 'Alpha'): -0.7631556448131983, ('Pre-endocrine', 'Beta'): -0.7453680508344683, ('Pre-endocrine', 'Delta'): -0.2887071506917505, ('Pre-endocrine', 'Epsilon'): -0.3417304090937336}
Total Mean: -0.5963613171862915
# In-cluster Coherence
{'Alpha': 0.75067365, 'Beta': 0.71837527, 'Delta': 0.5667012, 'Ductal': 0.86601305, 'Epsilon': 0.615985, 'Ngn3 high EP': 0.75801826, 'Ngn3 low EP': 0.86726254, 'Pre-endocrine': 0.7931749}
Total Mean: 0.7420254945755005
2
2
2
Keeping at most 100000 cells per cluster
Filtered out 20801 genes that are detected 20 counts (shared).
Extracted 6000 highly 

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Number of Leiden Clusters: 13
Maximal Number of Modules: 14
First training run ...
Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:22<00:00,  2.03s/it, v_num=1, elbo_train=2.39e+7]
Initializing second training run ...
Sampling global variables, sample: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.71it/s]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Second training run ...
Epoch 254/400:  63%|█████████████████████████████████████████████████████████████████████████████████████████████████▍                                                        | 253/400 [08:31<04:56,  2.01s/it, v_num=1, elbo_train=2.32e+7]